In [1]:
# Import pandas 
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
# Load the customer_data
customer_data = pd.read_csv('datasets/Customer_dataset.csv',
                            parse_dates=True,
                            infer_datetime_format=True)

# Load the app_purchases
app_purchases = pd.read_csv('datasets/inapp_purchases_dataset.csv',
                            parse_dates=True,
                            infer_datetime_format=True)

# Print the columns of customer data
print(customer_data.columns)

# Print the columns of app_purchases
print(app_purchases.columns)

Index(['uid', 'reg_date', 'device', 'gender', 'country', 'age'], dtype='object')
Index(['date', 'uid', 'sku', 'price'], dtype='object')


In [3]:
customer_data.head(3)

,uid,reg_date,device,gender,country,age
0,54030035.0,2017-06-29T00:00:00Z,and,M,USA,19
1,72574201.0,2018-03-05T00:00:00Z,iOS,F,TUR,22
2,64187558.0,2016-02-07T00:00:00Z,iOS,M,USA,16


In [4]:
app_purchases.head(3)

,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599


### Grouping datasets:

In [5]:
# Changing reg_date column type from string to datetime:
customer_data["reg_date"] = pd.to_datetime(customer_data["reg_date"])  

In [6]:
# Adding new colum as reg_date_1 which is changed type from timestapt to datetime(Y-M-D):
customer_data['reg_date_1'] = customer_data['reg_date'].dt.strftime('%Y-%m-%d')

In [7]:
# Merge on the 'uid' and 'date' field
uid_date_combined_data = app_purchases.merge(customer_data, left_on=['uid','date'],right_on=['uid','reg_date_1'], how='inner')

# Examine the results 
print(uid_date_combined_data.head())
print(len(uid_date_combined_data))

         date       uid             sku  price                  reg_date  \
0  2016-03-30  94055095    sku_four_599    599 2016-03-30 00:00:00+00:00   
1  2015-10-28  69627745     sku_one_199    199 2015-10-28 00:00:00+00:00   
2  2017-02-02  11604973  sku_seven_1499    499 2017-02-02 00:00:00+00:00   
3  2016-06-05  22495315    sku_four_599    599 2016-06-05 00:00:00+00:00   
4  2018-02-17  51365662     sku_two_299    299 2018-02-17 00:00:00+00:00   

  device gender country  age  reg_date_1  
0    iOS      F     BRA   16  2016-03-30  
1    and      F     BRA   18  2015-10-28  
2    and      F     USA   16  2017-02-02  
3    and      F     USA   19  2016-06-05  
4    iOS      M     TUR   16  2018-02-17  
35


### Parsing dates
In this exercise you will practice parsing dates in Python. While often data pulled from a database will be correctly formatted, other data sources can be less nice. Knowing how to properly parse dates is crucial to get the data in a workable format. For reference refer to https://strftime.org/ throughout this exercise to see date format to use.

In [8]:
# Provide the correct format for the date: 2016 March 01 01:56
date_data_four = "2016 March 01 01:56"
date_data_four = pd.to_datetime(date_data_four, format="%Y %B %d %H:%M")
print(date_data_four)

2016-03-01 01:56:00


In [9]:
# Provide the correct format for the date: Saturday January 27, 2017
date_data_one = "Saturday January 27, 2017"
date_data_one = pd.to_datetime(date_data_one, format="%A %B %d, %Y")
print(date_data_one)

2017-01-27 00:00:00


### Plotting time series data
In trying to boost purchases, we have made some changes to our introductory in-app purchase pricing. In this exercise, you will check if this is having an impact on the number of purchases made by purchasing users during their first week.

The dataset user_purchases has been joined to the demographics data and properly filtered. The column 'first_week_purchases' that is 1 for a first week purchase and 0 otherwise has been added. This column is converted to the average number of purchases made per day by users in their first week.

We will try to view the impact of this change by looking at a graph of purchases as described in the instructions.

In [10]:
uid_date_combined_data.head()

,date,uid,sku,price,reg_date,device,gender,country,age,reg_date_1
0,2016-03-30,94055095,sku_four_599,599,2016-03-30 00:00:00+00:00,iOS,F,BRA,16,2016-03-30
1,2015-10-28,69627745,sku_one_199,199,2015-10-28 00:00:00+00:00,and,F,BRA,18,2015-10-28
2,2017-02-02,11604973,sku_seven_1499,499,2017-02-02 00:00:00+00:00,and,F,USA,16,2017-02-02
3,2016-06-05,22495315,sku_four_599,599,2016-06-05 00:00:00+00:00,and,F,USA,19,2016-06-05
4,2018-02-17,51365662,sku_two_299,299,2018-02-17 00:00:00+00:00,iOS,M,TUR,16,2018-02-17


In [11]:
# Adding new column as first_week_purchases:
uid_date_combined_data['first_week_purchases'] = 

SyntaxError: invalid syntax (2908304873.py, line 2)

In [ ]:
# Group the data and aggregate first_week_purchases
uid_date_combined_data = uid_date_combined_data.groupby(by=['reg_date', 'uid']).agg({'first_week_purchases': ['sum']})

# Reset the indexes
app_purchases.columns = app_purchases.columns.droplevel(level=1)
app_purchases.reset_index(inplace=True)

# Find the average number of purchases per day by first-week users
app_purchases = app_purchases.groupby(by=['reg_date']).agg({'first_week_purchases': ['mean']})
app_purchases.columns = app_purchases.columns.droplevel(level=1)
app_purchases.reset_index(inplace=True)

# Plot the results
app_purchases.plot(x='reg_date', y='first_week_purchases')
plt.show()

### Pivoting our data
As you saw, there does seem to be an increase in the number of purchases by purchasing users within their first week. Let's now confirm that this is not driven only by one segment of users. We'll do this by first pivoting our data by 'country' and then by 'device'. Our change is designed to impact all of these groups equally.

The user_purchases data from before has been grouped and aggregated by the 'country' and 'device' columns. These objects are available in your workspace as user_purchases_country and user_purchases_device.

As a reminder, .pivot_table() has the following signature:

pd.pivot_table(data, values, columns, index)

In [ ]:
# Pivot the data
country_pivot = pd.pivot_table(user_purchases_country, values=['first_week_purchases'], columns=['country'], index=['reg_date'])
print(country_pivot.head())

In [ ]:
# Pivot the data
device_pivot = pd.pivot_table(user_purchases_device, values=['first_week_purchases'], columns=['device'], index=['reg_date'])
print(device_pivot.head())

### Examining the different cohorts
To finish this lesson, you're now going to plot by 'country' and then by 'device' and examine the results. Hopefully you will see the observed lift across all groups as designed. This would point to the change being the cause of the lift, not some other event impacting the purchase rate.

In [ ]:
# Plot the average first week purchases for each country by registration date
country_pivot.plot(x='reg_date', y=['USA', 'CAN', 'FRA', 'BRA', 'TUR', 'DEU'])
plt.show()

In [ ]:
# Plot the average first week purchases for each device by registration date
device_pivot.plot(x='reg_date', y=['and', 'iOS'])
plt.show()